### PCA概述
    作用:高维度数据 降维度,
    1.便于显示
    2.可以剔除数据中的 "干扰"

### 数据源
    1.矩阵格式,2个字段(id,data)
    2.自变量(data)必须是数组
    
    
    DROP TABLE IF EXISTS mat;
    CREATE TABLE mat (id integer,
                      row_vec double precision[]
    #3维数据                  );
    INSERT INTO mat VALUES
    (1, '{1,2,3}'),
    (2, '{2,1,2}'),
    (3, '{3,2,1}');

## 训练 生成 "投影轴"

    DROP TABLE IF EXISTS result_table, result_table_mean;
    SELECT madlib.pca_train('mat',             -- 输入的数据表(矩阵)
                            'result_table',    -- 输出的表名
                            'id',              -- 输入表的主键
                             2);               -- 降到多少维度
    SELECT * FROM result_table ORDER BY row_id;
<img src='img/pca1.png'/>

## 数据投影到 "投影轴"

    DROP TABLE IF EXISTS residual_table, result_summary_table, out_table;
    SELECT madlib.pca_project( 'mat',  --需要投影的数据表,格式如上
                               'result_table',--投影轴的 表
                               'out_table', --输出表
                               'id', --输入的主键
                               'residual_table',--近似参与信息表
                               'result_summary_table'
                               );
    SELECT * FROM out_table ORDER BY row_id;
<img src='img/pca2.png' />

# KMean 聚类

    根据特征,把数据聚合成为K个组
    1.聚类算法使用之前 一般需要对数据做"预处理",PCA降为,列缩放到[0,1]区间,这里省略....

### 1.数据格式(矩阵格式)
    DROP TABLE IF EXISTS km_sample;
    CREATE TABLE km_sample(pid int, points double precision[]);
    INSERT INTO km_sample VALUES
    (1,  '{14.23, 1.71, 2.43, 15.6, 127, 2.8, 3.0600, 0.2800, 2.29, 5.64, 1.04, 3.92, 1065}'),
    (2,  '{13.2, 1.78, 2.14, 11.2, 1, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050}'),
    (3,  '{13.16, 2.36,  2.67, 18.6, 101, 2.8,  3.24, 0.3, 2.81, 5.6799, 1.03, 3.17, 1185}'),
    (4,  '{14.37, 1.95, 2.5, 16.8, 113, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480}'),
    (5,  '{13.24, 2.59, 2.87, 21, 118, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735}'),
    (6,  '{14.2, 1.76, 2.45, 15.2, 112, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450}'),
    (7,  '{14.39, 1.87, 2.45, 14.6, 96, 2.5, 2.52, 0.3, 1.98, 5.25, 1.02, 3.58, 1290}'),
    (8,  '{14.06, 2.15, 2.61, 17.6, 121, 2.6, 2.51, 0.31, 1.25, 5.05, 1.06, 3.58, 1295}'),
    (9,  '{14.83, 1.64, 2.17, 14, 97, 2.8, 2.98, 0.29, 1.98, 5.2, 1.08, 2.85, 1045}'),
    (10, '{13.86, 1.35, 2.27, 16, 98, 2.98, 3.15, 0.22, 1.8500, 7.2199, 1.01, 3.55, 1045}');

### 2.训练生成 组 的"中心"

    DROP TABLE IF EXISTS km_result;
    -- Run kmeans algorithm
    
    
    CREATE TABLE km_result AS  --这里把中心 保存到了 km_result;表中
    SELECT * FROM madlib.kmeanspp(
        'km_sample',--输入表 
        'points', --聚类的自变量名
         2,--聚类数K
        'madlib.squared_dist_norm2',--度量函数,有dist_norm1: ,dist_norm2,squared_dist_norm2,dist_angle四种
        'madlib.avg', --中心聚合函数
         20,--算法迭代次数,应该与数据量成比例 
         0.001);
     #生成了 km_result 表,查看可以看到中心,(中心可用于显示每个分类的判定依据)
<img src='img/kmean1.png' />

## 3.对"新数据" 进行分类

    SELECT 
        (madlib.closest_column(centroids, points)).column_id --分类的结果,传入要分类的列,中心
    FROM 
         km_sample, --输入表 
         km_result; --中心表